In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Load the dataset
age_labels = np.load("./Dataset/imdb-wiki-image-npy/age.npy")
gender_labels = np.load("./Dataset/imdb-wiki-image-npy/gender.npy")
image_data = np.load("./Dataset/imdb-wiki-image-npy/imdb-wiki-image-data.npy")

# Check the shape of the arrays
print("Age Labels Shape:", age_labels.shape)
print("Gender Labels Shape:", gender_labels.shape)
print("Image Data Shape:", image_data.shape)

Age Labels Shape: (36356,)
Gender Labels Shape: (36356,)
Image Data Shape: (36356, 10000)


In [3]:
# Reshape image data to original form (assuming images are square)
image_size = int(np.sqrt(image_data.shape[1]))
image_data = image_data.reshape(-1, image_size, image_size, 1)

# Normalize pixel values to range [0, 1]
image_data = image_data.astype('float32') / 255.0

# Check the updated shape
print("Updated Image Data Shape:", image_data.shape)


Updated Image Data Shape: (36356, 100, 100, 1)


In [15]:
# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Add dropout layer with dropout rate of 0.5
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  
    layers.Dense(1)  # Output layer for age prediction
])

In [16]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mape'])

# Print model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 49, 49, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 10, 10, 128)      

In [17]:
# Define training and validation data
X_train, X_val = image_data[:29000], image_data[29000:]
y_train, y_val = age_labels[:29000], age_labels[29000:]

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, mae, mape = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation MAE:", mae)
print("Validation MAPE:", mape)


Epoch 1/10
907/907 [==============================] - 180s 197ms/step - loss: 168.5708 - mae: 10.0821 - mape: 34.9244 - val_loss: 119.9932 - val_mae: 8.6259 - val_mape: 31.8803
Epoch 2/10
907/907 [==============================] - 180s 198ms/step - loss: 146.6266 - mae: 9.4368 - mape: 33.0590 - val_loss: 129.9716 - val_mae: 9.2937 - val_mape: 36.7784
Epoch 3/10
907/907 [==============================] - 193s 213ms/step - loss: 137.3920 - mae: 9.1084 - mape: 32.0011 - val_loss: 105.8906 - val_mae: 8.1052 - val_mape: 30.7599
Epoch 4/10
907/907 [==============================] - 221s 244ms/step - loss: 126.0402 - mae: 8.7313 - mape: 30.6347 - val_loss: 110.0553 - val_mae: 8.4438 - val_mape: 33.3262
Epoch 5/10
907/907 [==============================] - 216s 238ms/step - loss: 119.1065 - mae: 8.4625 - mape: 29.5490 - val_loss: 94.1944 - val_mae: 7.4294 - val_mape: 26.2065
Epoch 6/10
907/907 [==============================] - 217s 239ms/step - loss: 111.7693 - mae: 8.2336 - mape: 28.7494 - v

In [18]:
# Save the trained model
model.save("model_L80_M6_mape24.h5")